<a href="https://colab.research.google.com/github/oldper/LLM/blob/main/faq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese', num_labels=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def encode_question(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [ ]:
def predict(input_question, source, data):
  source_vectors = {}
  source = [str(s) for s in source]
  for id in source:
      if id in data:
          source_vectors[id] = [encode_question(item['question']) for item in data[id]]
  input_vector = encode_question(input_question)

  most_similar_question = None
  most_similar_id = None
  highest_similarity = -1

  for id, vectors in source_vectors.items():
    for vector, item in zip(vectors, data[id]):
      question = item['question']
      score = cosine_similarity([input_vector], [vector])[0][0]
      if score > highest_similarity:
          highest_similarity = score
          most_similar_question = question
          most_similar_id = id

  return most_similar_id


In [ ]:
import json
import tqdm
with open('pid_map_content.json', 'r') as f:
  data = json.load(f)

with open('questions_example.json', 'r') as f:
  questions = json.load(f)

arr = []
for q in questions['questions']:
  if q['category'] == 'faq':
    arr.append(predict(q['query'], q['source'], data))


In [ ]:
with open('ground_truths_example.json', 'r') as f:
  gt = json.load(f)

ans_arr = []
for ans in gt['ground_truths']:
  if ans['category'] == 'faq':
    ans_arr.append(ans['retrieve'])

ans_arr

[558,
 104,
 63,
 15,
 294,
 224,
 540,
 105,
 283,
 611,
 76,
 403,
 509,
 279,
 54,
 4,
 92,
 554,
 610,
 20,
 414,
 415,
 527,
 14,
 604,
 1,
 365,
 436,
 283,
 5,
 600,
 334,
 243,
 328,
 28,
 504,
 359,
 339,
 194,
 141,
 410,
 173,
 144,
 225,
 564,
 0,
 87,
 242,
 441,
 221]

In [ ]:
c = 0
arr = [int(a) for a in arr]
for i in range(len(arr)):
  if arr[i] == ans_arr[i]:
    c+=1
  else:
    print(i)

print("Accuracy: ", c/len(arr))

8
34
Accuracy:  0.96
